In [3]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder, PowerTransformer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import randint, uniform
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shap


### Last inn data

In [4]:
# Last inn data
data = pd.read_csv('Data/data_grouped.csv') # Endre til aktuell mappe
# lag nye navn fra-sone-6 og til-sone-6 til fra-sone og til-sone
data = data.rename(columns={'fra-sone-6': 'fra-sone', 'til-sone-6': 'til-sone', 'fri_hen_priv': 'fritid'})
# Lag en hjelpe-tabell med unike soner og befolkning
pop_lookup = data[['fra-sone', 'befolkning']].drop_duplicates()
pop_lookup.columns = ['sone', 'til_befolkning']

# Slå opp befolkning for to_zone basert på from_zone-informasjon
df = data.merge(pop_lookup, left_on='til-sone', right_on='sone', how='left')
df = df.drop(columns='sone')

data = df

In [36]:
xgb_data = data.copy()

label_encoder_fra = LabelEncoder()
xgb_data['fra-sone-encoded'] = label_encoder_fra.fit_transform(xgb_data['fra-sone'])

label_encoder_til = LabelEncoder()
xgb_data['til-sone-encoded'] = label_encoder_til.fit_transform(xgb_data['til-sone'])

label_encoder_tid = LabelEncoder()
xgb_data['tid-encoded'] = label_encoder_tid.fit_transform(xgb_data['tid'])

X_1 = xgb_data[['fra-sone-encoded', 'til-sone-encoded', 'tid-encoded', 'befolkning', 'to_befolkning', 'arbeidsplasser', 'handel', 'fri_hen_priv', 'distanse']]
# Definer input- og output-variabler
X = X_1.values
y = xgb_data['reiser'].values  # Ingen log-transformasjon!
#y = np.log1p(y) # ved test av log-trans


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Transformer y_train og y_test separat
#pt_y = PowerTransformer()
#y_train = pt_y.fit_transform(y_train_org.reshape(-1, 1)).flatten()
#y_test = pt_y.transform(y_test_org.reshape(-1, 1)).flatten()

feature_names = X_1.columns.tolist()

# Lag DataFrame for X_test
X_test_df = pd.DataFrame(X_test, columns=feature_names)

y_test_df = pd.DataFrame(y_test_org, columns=['reiser']) 
# Sett sammen X_test og y_test
t_set = pd.concat([X_test_df.reset_index(drop=True), y_test_df.reset_index(drop=True)], axis=1)

### Tuning av hyperparamtere

In [ ]:

param_dist = {
    'n_estimators': randint(50, 500),  # Antall trær
    'learning_rate': uniform(0.01, 0.3),  # Læringsrate mellom 0.01 og 0.3
    'max_depth': randint(3, 15),  # Dybde på trærne
    'min_child_weight': randint(1, 10),  # Minimum vekt per bladnode
    'subsample': uniform(0.5, 0.5),  # Andel av data brukt per tre
    'colsample_bytree': uniform(0.5, 0.5)  # Andel av features brukt per tre
}

xgb = XGBRegressor(objective='reg:squarederror', random_state=42, n_jobs=-1)

# Random search leter etter optimale paramtere
random_search = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_dist,
    n_iter=20,  # Test 20 tilfeldige kombinasjoner for lav beregningstid
    cv=3,  # 3-fold cross-validation
    scoring='neg_mean_squared_error',
    verbose=2,
    random_state=42,
    n_jobs=-1
)

In [ ]:
random_search.fit(X_train, y_train)

### Lagrede paramtere for ulike formål

In [26]:
# uten transform
best_params = {'colsample_bytree': 0.5202167947692157, 'learning_rate': 0.22319886690573623, 'max_depth': 12, 'min_child_weight': 3, 'n_estimators': 112, 'subsample': 0.9478817978367597}

In [30]:
# log-transform
best_params = {'colsample_bytree': np.float64(0.6404672548436904), 'learning_rate': np.float64(0.17280882494747454), 'max_depth': 11, 'min_child_weight': 1, 'n_estimators': 394, 'subsample': np.float64(0.9934434683002586)}

In [ ]:
# PT
best_params = {'colsample_bytree': np.float64(0.49665415678116653), 'learning_rate': np.float64(0.17280882494747454), 'max_depth': 11, 'min_child_weight': 1, 'n_estimators': 394, 'subsample': np.float64(0.9908208556203622)}

### Modeller

In [ ]:
# ta tiden på trening:
import time
start_time = time.time()


xgb_best = XGBRegressor(**best_params, objective='reg:squarederror', random_state=42, n_jobs=-1)

xgb_best.fit(X_train, y_train)

end_time = time.time()
training_time = end_time - start_time
print(f"Treningstid: {training_time:.2f} sekunder")

In [ ]:
tid1 = time.time()

y_pred = xgb_best.predict(X_test)
tid2 = time.time()
print(f"Gjør prediksjoner: {tid2 - tid1:.2f} sekunder")

y_pred_real = y_pred 
y_test_real = y_test 

#np.expm1(y_pred)
#np.expm1(y_test)

#y_pred_real = pt_y.inverse_transform(y_pred.reshape(-1, 1)).ravel()
#y_test_real = pt_y.inverse_transform(y_test.reshape(-1, 1)).ravel()


mse = mean_squared_error(y_test_real, y_pred_real)
mae = mean_absolute_error(y_test_real, y_pred_real)
r2 = r2_score(y_test_real, y_pred_real)

### Resulater

In [ ]:
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"R-squared (R2): {r2:.2f}")

In [ ]:
#MAPE
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")

In [ ]:

errors = abs(y_pred - y_test)

mape = 100 * (errors / y_test)

median_mape = np.median(mape)
print(f"Median MAPE: {median_mape:.2f}%")


### Plot av residualer og predikerte verdier

In [ ]:
# Plot true vs predicted values
plt.figure(figsize=(10, 6))
plt.scatter(y_test_real, y_pred, alpha=0.5, label='Predicted vs True')
plt.plot([min(y_test_real), max(y_test_real)], [min(y_test_real), max(y_test_real)], '--r', label='Perfect Prediction')
plt.xlabel('True Values (y_test)')
plt.ylabel('Predicted Values (y_pred)')
plt.title('True vs Predicted Values - XGB')
plt.legend()
plt.grid(True)
# Filter out actual values greater than 100,000
mask = y_test_real <= 10000
filtered_y_test = y_test_real[mask]
filtered_y_pred = y_pred_real[mask]

# Plot filtered true vs predicted values
plt.figure(figsize=(10, 6))
plt.scatter(filtered_y_test, filtered_y_pred, alpha=0.5, label='Predicted vs True')
plt.plot([min(filtered_y_test), max(filtered_y_test)], [min(filtered_y_test), max(filtered_y_test)], '--r', label='Perfect Prediction')
plt.xlabel('True Values (y_test)')
plt.ylabel('Predicted Values (y_pred)')
plt.title('True vs Predicted Values (Filtered)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Plot residuals

residuals = y_test - y_pred
plt.figure(figsize=(10, 5))
plt.scatter(y_test, residuals, alpha=0.5)
plt.xlabel('Antall reiser (Faktiske verider)')
plt.ylabel('Residual')
plt.axhline(y=0, color='black', linewidth=2)
plt.title('Residualer mot faktiske verdier')
plt.grid(True)
plt.show()

In [ ]:
# Beregn residualer
residuals = y_test - y_pred

# Filtrer der y_test ≤ 40000
mask = y_test_org <= 40000
y_test_filtered = y_test_org[mask]
residuals_filtered = residuals[mask]

# Plot
plt.figure(figsize=(10, 5))
plt.scatter(y_test_filtered, residuals_filtered, alpha=0.5)
plt.xlabel('Antall reiser (Faktisk verdier)')
plt.ylabel('Residual')
plt.axhline(y=0, color='black', linewidth=2)
plt.title('Residualer mot Faktisk verdi (≤ 40 000)')
plt.grid(True)
plt.show()

### Modellusikkerhet

In [ ]:
n_models = 20
seeds = np.random.randint(0, 10000, n_models)

all_predictions = []

for seed in seeds:
    rf = XGBRegressor(**best_params, objective='reg:squarederror', random_state=seed, n_jobs=-1)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    all_predictions.append(y_pred)

all_predictions = np.array(all_predictions)

# Beregn gjennomsnitt og standardavvik
mean_pred = all_predictions.mean(axis=0)
std_pred = all_predictions.std(axis=0)
rel_std_pred = std_pred / (mean_pred + 1e-6)  # +1e-6 for å unngå deling på null

# Eksempel: Vis relativ usikkerhet for de 10 første testprøvene
print("Relativ standardavvik:", rel_std_pred[:10])
print("Gjennomsnittlig prediksjon:", mean_pred[:10])


In [ ]:
# Gjennomsnittlig standardavvik
mean_std = rel_std_pred.mean()
print(f"Gjennomsnittlig standardavvik: {mean_std:.2f}")
meanst = std_pred.mean()
print(f"Gjennomsnittlig standardavvik: {meanst:.2f}")

### Overtilpasning

In [ ]:
# sammenlign resultater med prediksjonser på testsettet med prediksjoner på treningssettet
y_train_pred = xgb_best.predict(X_train)
y_test_pred = xgb_best.predict(X_test)

mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)

mae_train = mean_absolute_error(y_train, y_train_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)

r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)

print("\nModellens ytelse med optimaliserte hyperparametere:")
print(f"Mean Squared Error (MSE) - Treningssett: {mse_train:.2f}")
print(f"Mean Squared Error (MSE) - Testsett: {mse_test:.2f}")
print(f"Mean Absolute Error (MAE) - Treningssett: {mae_train:.2f}")
print(f"Mean Absolute Error (MAE) - Testsett: {mae_test:.2f}")
print(f"R-squared (R2) - Treningssett: {r2_train:.2f}")
print(f"R-squared (R2) - Testsett: {r2_test:.2f}")

### SHAP

In [ ]:
explainer = shap.TreeExplainer(xgb_best)
# find x_sample, only some of the test-data
X_sample = X_test[:1000]
shap_values_sample = explainer.shap_values(X_sample)

In [ ]:
# Velg kun de numeriske feature-navnene
numerical_features = ['befolkning', 'to_befolkning','arbeidsplasser', 'handel', 'fritid', 'distanse']

# Finn indeksene til de numeriske feature-navnene i den opprinnelige listen
feature_indices = [i for i, name in enumerate(['fra-sone', 'til-sone', 'tid', 'befolkning', 'to_befolkning','arbeidsplasser', 'handel', 'fritid', 'distanse']) if name in numerical_features]

# Filtrer SHAP-verdiene for kun de numeriske features
shap_values_filtered = shap_values_sample[:, feature_indices]
X_filtered = X_sample[:, feature_indices]
numerical_features = ['befolkning', 'befolkning i til-sone', 'arbeidsplasser', 'handel', 'fritid', 'distanse']

# Lag summary-plot uten de kategoriske variablene
shap.summary_plot(shap_values_filtered, X_filtered, max_display=10,
                  show=False, feature_names=numerical_features)
plt.title('SHAP Summary Plot - XGB')
plt.xlim(-1000, 1000)
plt.show()



### Kryssvalidasjon

In [ ]:
# Cross-validation
from sklearn.model_selection import cross_val_score, KFold

kf = KFold(n_splits=5, shuffle=True, random_state=42)

scores = cross_val_score(xgb_best, X, y, cv=kf, scoring='neg_mean_squared_error')

mse_scores = -scores

# Lagre resultater fra ulike folder
mae_scores = []
r2_scores = []
mape_scores = []

for train_idx, test_idx in kf.split(X):
    X_train_fold, X_test_fold = X[train_idx], X[test_idx]
    y_train_fold, y_test_fold = y[train_idx], y[test_idx]

    xgb_best.fit(X_train_fold, y_train_fold)
    y_pred_fold = xgb_best.predict(X_test_fold)

    mae_scores.append(mean_absolute_error(y_test_fold, y_pred_fold))
    r2_scores.append(r2_score(y_test_fold, y_pred_fold))
    mape_scores.append(np.mean(np.abs((y_test_fold - y_pred_fold) / y_test_fold)) * 100)


print(f"MAE scores: {mae_scores}")
print(f"Mean MAE: {np.mean(mae_scores):.2f}")
print(f"Standard deviation MAE: {np.std(mae_scores):.2f}")

print(f"R2 scores: {r2_scores}")
print(f"Mean R2: {np.mean(r2_scores):.2f}")
print(f"Standard deviation R2: {np.std(r2_scores):.2f}")

print(f"MAPE scores: {mape_scores}")
print(f"Mean MAPE: {np.mean(mape_scores):.2f}%")
print(f"Standard deviation MAPE: {np.std(mape_scores):.2f}%")

### Analyse av tid og rom

In [ ]:
# test-data med kun reiser med antall reiser > 10000
test_data = t_set[t_set['reiser'] < 1000]
X_test = test_data[['fra-sone-encoded', 'til-sone-encoded', 'tid-encoded', 'befolkning', 'to_befolkning','arbeidsplasser', 'handel', 'fri_hen_priv', 'distanse']].values
y_test = test_data['reiser'].values

y_pred = xgb_best.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\nModellens ytelse på testsettet:")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"R-squared (R2): {r2:.2f}")
errors = abs(y_pred - y_test)

mape = 100 * (errors / y_test)

median_mape = np.median(mape)
print(f"Median MAPE: {median_mape:.2f}%")

mean_mape = np.mean(mape)
print(f"Mean MAPE: {mean_mape:.2f}%")

In [ ]:
# Test-data med kun reiser med antall reiser < 1000 < 10000
test_data = t_set[(t_set['reiser'] < 10000) & (t_set['reiser'] >= 1000)]
X_test = test_data[['fra-sone-encoded', 'til-sone-encoded', 'tid-encoded', 'befolkning', 'to_befolkning', 'arbeidsplasser', 'handel', 'fri_hen_priv', 'distanse']].values
y_test = test_data['reiser'].values

y_pred = xgb_best.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\nModellens ytelse på testsettet:")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"R-squared (R2): {r2:.2f}")
errors = abs(y_pred - y_test)

mape = 100 * (errors / y_test)

median_mape = np.median(mape)
print(f"Median MAPE: {median_mape:.2f}%")

mean_mape = np.mean(mape)
print(f"Mean MAPE: {mean_mape:.2f}%")

In [ ]:
# test-data med kun reiser med antall reiser > 10000
test_data = t_set[t_set['reiser'] > 10000]
X_test = test_data[['fra-sone-encoded', 'til-sone-encoded', 'tid-encoded', 'befolkning', 'to_befolkning','arbeidsplasser', 'handel', 'fri_hen_priv', 'distanse']].values
y_test = test_data['reiser'].values

y_pred = xgb_best.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\nModellens ytelse på testsettet:")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"R-squared (R2): {r2:.2f}")
errors = abs(y_pred - y_test)

mape = 100 * (errors / y_test)

median_mape = np.median(mape)
print(f"Median MAPE: {median_mape:.2f}%")

mean_mape = np.mean(mape)
print(f"Mean MAPE: {mean_mape:.2f}%")

In [ ]:
# Test-data med kun punkter der tid = 'UR' # Endre til 1 og 0 for Helg og rush
test_data = t_set[t_set['tid-encoded'] == 2]
X_test = test_data[['fra-sone-encoded', 'til-sone-encoded', 'tid-encoded', 'befolkning', 'to_befolkning', 'arbeidsplasser', 'handel', 'fri_hen_priv', 'distanse']].values
y_test = test_data['reiser'].values

y_pred = xgb_best.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\nModellens ytelse på testsettet:")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"R-squared (R2): {r2:.2f}")
errors = abs(y_pred - y_test)

mape = 100 * (errors / y_test)

median_mape = np.median(mape)
print(f"Median MAPE: {median_mape:.2f}%")

mean_mape = np.mean(mape)
print(f"Mean MAPE: {mean_mape:.2f}%")

In [ ]:
# test_data der befolkning er under 500
test_data = t_set[t_set['befolkning'] < 500] # endre til over 500 for å sjekke større befolkning
X_test = test_data[['fra-sone-encoded', 'til-sone-encoded', 'tid-encoded', 'befolkning', 'to_befolkning', 'arbeidsplasser', 'handel', 'fri_hen_priv', 'distanse']].values
y_test = test_data['reiser'].values

y_pred = xgb_best.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\nModellens ytelse på testsettet:")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"R-squared (R2): {r2:.2f}")
errors = abs(y_pred - y_test)

mape = 100 * (errors / y_test)

median_mape = np.median(mape)
print(f"Median MAPE: {median_mape:.2f}%")

mean_mape = np.mean(mape)
print(f"Mean MAPE: {mean_mape:.2f}%")